In [ ]:
%matplotlib inline

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import torch as pt
import time
import sys

sys.path.insert(0, '..')

from NNsolver import NNSolver
from NNarchitectures import DenseNet_g
from problems import HJB
from utilities import get_X_process, plot_NN_evaluation, compute_PDE_loss

%load_ext autoreload
%autoreload 2

device = pt.device('cuda')

In [ ]:
T = 1.0
problem = HJB(d=100, T=T)
K = 2000
K_batch = 2000
print_every = 500
delta_t = 0.01
sq_delta_t = pt.sqrt(pt.tensor(delta_t))
N = int(np.ceil(T / delta_t))
gradient_steps = (N - 1) * [5000] + [40000]
learning_rates = [0.0001] * (N - 1) + [0.0003]


model = NNSolver(problem, 'HJB', learning_rates=learning_rates, gradient_steps=gradient_steps, NN_class=DenseNet_g, K=K, 
                 K_batch=K_batch, delta_t=delta_t, print_every=print_every, method='implicit')

model.Y_n = [DenseNet_g(problem.d, 1, lr=learning_rates[n], problem=problem).to(device) for n in range(N)] + [problem.g]

In [ ]:
model.train()

In [ ]:
fig = plot_NN_evaluation(model, n=N-4, reference_solution=False, Y_0_true=4.589992)

### compute reference loss

In [ ]:
problem.modus = 'np'
X, xi = get_X_process(problem, K, delta_t, seed=44)
problem.modus = 'pt'

In [ ]:
mean_L2_error = []
rel_error = []
for n in range(N + 1):
    Y_ref_traj = []
    for k in range(K):
        X_T_t = X[n, k, :][np.newaxis, :] + np.sqrt(2 * (problem.T - n * delta_t)) * np.random.randn(1000, problem.d)
        Y_ref_traj.append(-np.log(np.mean(1 / (0.5 + 0.5 * np.sum(X_T_t**2, 1)))))
    mean_L2_error.append(np.mean((model.Y_n[n](pt.tensor(X[n, :, :]).to(device).float()).squeeze().detach().cpu().numpy() 
                             - np.array(Y_ref_traj))**2))
    rel_error.append(np.mean(np.abs((model.Y_n[n](pt.tensor(X[n, :, :]).to(device).float()).squeeze().detach().cpu().numpy() 
                             - np.array(Y_ref_traj)) / np.array(Y_ref_traj))))

In [ ]:
rel_loss_TT = [8.121173074554055e-05, 8.029931663964015e-05, 8.880645396581369e-05, 9.466421304392999e-05, 0.00011042957207234186, 0.00011660206067459944, 0.00012578889070492927, 0.0001428875247914625, 0.0001553205323085007, 0.00017438877989229315, 0.00018612285347568562, 0.0001965697440503174, 0.00021145305254621338, 0.00022677254850598537, 0.00023769700643821675, 0.0002461860689543879, 0.00026887144417384767, 0.000277357110486139, 0.0002967044728136537, 0.00030039832989579087, 0.0003162312928035658, 0.0003345496370344649, 0.00035224141880109937, 0.00035538212858615627, 0.00037102812152886306, 0.00038530874636507126, 0.00038329595544280123, 0.00039928067196532855, 0.00041522405750533394, 0.0004306944527137073, 0.0004504872292143065, 0.00046903009137194045, 0.0004844640969781943, 0.0004772465854929323, 0.0004774195803142547, 0.00048549657113008493, 0.0004978592159402976, 0.0004916856071594701, 0.0004978473468799982, 0.0005163065694726725, 0.0005081884577479616, 0.0005193093144934012, 0.0005346528173871246, 0.0005406828357991653, 0.0005329697027719289, 0.0005469363463554905, 0.0005510421137367728, 0.0005517801132675107, 0.0005379024589511074, 0.0005385901161267378, 0.0005292451155714886, 0.0005363635423544875, 0.0005356570151948649, 0.0005214576102224184, 0.0005288335194964544, 0.0005308271139237318, 0.0005336972516919941, 0.0005318622513128217, 0.000522477694743566, 0.0005142959948745291, 0.0005117028428997801, 0.0005126887528943382, 0.0005092639245426013, 0.0004949676853034019, 0.0004989788516986965, 0.0004884434495776661, 0.00048057027272907624, 0.00046779426072851425, 0.00046185755696753127, 0.0004538633687977456, 0.00044270958704841406, 0.0004401859475577464, 0.00042938225992496216, 0.0004237283147014601, 0.0004192248594978464, 0.00040472137664940457, 0.0003894203542316912, 0.0003704973157274936, 0.00036169541878998363, 0.00035943626167213137, 0.0003445174436644364, 0.0003356061349342642, 0.000324839073183233, 0.0003074467763586861, 0.00029926106006017986, 0.00027780143923662216, 0.0002616419048375529, 0.0002428327007970543, 0.00023659832648419924, 0.0002233157081626156, 0.00020659219099715807, 0.00019307421196539405, 0.0001741173116197309, 0.00015393608113303554, 0.00013958701281246443, 0.00011959844900492289, 0.00010219599378600147, 8.668911077835847e-05, 6.684761159996627e-05, 4.828152127604531e-05, 2.7486718544441972e-05]

fig, ax = plt.subplots(1, 1, figsize=(5, 3.5))
ax.plot(np.linspace(0, problem.T - delta_t, N), rel_error[:-1], label=r'NN implicit')
ax.plot(np.linspace(0, problem.T - delta_t, N), rel_loss_TT[:-1], label='TT implicit')
ax.legend()
ax.set_xlabel(r'$t$')
ax.set_title('Mean relative error over time');
ax.set_yscale('log')
#fig.savefig('img/HJB_100_mean_relative_error_over_time_2.pdf')

In [ ]:
np.mean(mean_L2_error)

In [ ]:
problem.modus = 'pt'
pde_loss_eval = compute_PDE_loss(problem, delta_t=0.01, K=K, Y_n=model.Y_n, vfun=None, testOde=None, seed=44, print_every=10)

In [ ]:
np.mean(pde_loss_eval)